In [1]:
import os
%load_ext autoreload
%autoreload 2
os.chdir("..")
os.chdir("..")
os.chdir("..")

In [2]:
import json
from numpy import random

def open_json(filepath):
    try:
        with open(filepath, 'r', encoding="utf-8") as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None
    
list_data_words = open_json('database/vocabulary/study_word_list.json')
random.shuffle(list_data_words)



In [17]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random
import pygame
import os
import string
from datetime import datetime
from pathlib import Path
from app.utils.save_data import save_game_data, del_game_task
from app.stats_words.analyzer import WordStatsAnalyzer

# Inicializa áudio
pygame.mixer.init()

def highlight_letters(user_input, shuffled_word, text_widget):
    text_widget.config(state='normal')
    text_widget.delete("1.0", tk.END)
    for letter in shuffled_word:
        if letter in user_input:
            text_widget.insert(tk.END, letter, "green")
        else:
            text_widget.insert(tk.END, letter)
    text_widget.tag_config("green", foreground="green")
    text_widget.config(state='disabled')

# Embaralhar palavra preservando pontuação
def shuffle_word(word):
    # Remove pontuação do núcleo da palavra
    core = ''.join([c for c in word if c.isalpha()])
    if len(core) <= 1:
        return word
    shuffled = list(core)
    while True:
        random.shuffle(shuffled)
        if ''.join(shuffled).lower() != core.lower():
            break
    shuffled_word = ''.join(shuffled)
    # Reconstrói a palavra com o mesmo sufixo de pontuação (se houver)
    suffix = ''.join([c for c in word if not c.isalpha()])
    return shuffled_word + suffix

# App principal
class WordShuffleGame(tk.Frame):
    def __init__(self, parent=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent

        self.entries:list[dict[str, str]] = []
        self.audio_path = ""

        # Frame principal
        self.frame = tk.Frame(self)
        self.frame.pack(padx=20, pady=20, fill="both", expand=True)

        # Frame para botões
        self.btn_frame = tk.Frame(self)
        self.btn_frame.pack(pady=10)

        # self.submit_btn = tk.Button(self.btn_frame, text="✅ Verificar", command=self.check_answers)
        # self.submit_btn.pack(side="left", padx=5)

        self.next_btn = tk.Button(self.btn_frame, text="➡ Próxima", command=self.load_sentence, state='disabled')
        self.next_btn.pack(side="left", padx=5)

        self.play_audio_btn = tk.Button(self.btn_frame, text="🔊 Ouvir Áudio", command=self.play_audio)
        self.play_audio_btn.pack(side="left", padx=5)

        # Botão Editar
        btn_editar = tk.Button(
            self.btn_frame,
            text="✏️ Editar",
            font=("Arial", 10),
            width=10,
            command=lambda: self.editar_json(dict_info_words=self.dict_info_words)
        )
        btn_editar.pack(side="left", padx=5)


        self.id_game = self.gerar_hash_id()

        self.clicks_on_guess = {}

        self.word_stats_analyzer = WordStatsAnalyzer(list_game_name=["game_data_hangman", "game_data_word_shuffle_game"])

        self.load_sentence()

    def editar_json(self, dict_info_words):
        SMALL_FONT = ("Arial", 10)  # ou use a fonte que desejar
        path_base = Path(dict_info_words.get("path", ""))
        path_text_json = path_base / "text_v2.json"
        path_image = path_base / "image_text.jpg"

        if not path_text_json.exists():
            messagebox.showerror("Erro", f"Arquivo não encontrado: {path_text_json}")
            return

        try:
            with open(path_text_json, 'r', encoding="utf-8") as f:
                json_data = json.load(f)
        except Exception as e:
            messagebox.showerror("Erro ao abrir JSON", str(e))
            return

        # Cria nova janela
        editor_window = tk.Toplevel(self)
        editor_window.title("Editar texto")

        current_row = 0

        # Exibe imagem no topo (se existir)
        if path_image.exists():
            try:
                img = Image.open(path_image).resize((200, 200))
                photo = ImageTk.PhotoImage(img)

                img_label = tk.Label(editor_window, image=photo)
                img_label.image = photo  # evitar que o garbage collector apague
                img_label.grid(row=current_row, column=0, columnspan=2, pady=10)
                current_row += 1
            except Exception as e:
                messagebox.showwarning("Imagem", f"Erro ao carregar imagem: {e}")

        entries = {}

        for key, value in json_data.items():
            tk.Label(editor_window, text=key, font=SMALL_FONT).grid(row=current_row, column=0, padx=10, pady=5, sticky="e")
            entry = tk.Entry(editor_window, width=60)
            entry.insert(0, value)
            entry.grid(row=current_row, column=1, padx=10, pady=5, sticky="w")
            entries[key] = entry
            current_row += 1

        delete_var = tk.BooleanVar(value=False)
        delete_check = tk.Checkbutton(
            editor_window,
            text="Excluir tarefa do banco de dados",
            variable=delete_var,
            font=SMALL_FONT
        )
        delete_check.grid(row=current_row, column=0, columnspan=2, pady=(5, 0))
        current_row += 1

        def salvar_alteracoes():
            for key in json_data:
                json_data[key] = entries[key].get()

            try:
                with open(path_text_json, 'w', encoding="utf-8") as f:
                    json.dump(json_data, f, ensure_ascii=False, indent=4)
                # Só apaga do banco se o checkbox estiver marcado
                if delete_var.get():
                    from app.utils.save_data import del_game_task
                    del_game_task(
                        id_game_task=self.id_game_task,
                        game_name="word_shuffle_game"
                    )
                messagebox.showinfo("Sucesso", "Arquivo salvo com sucesso.")
                editor_window.destroy()
                self.load_sentence()
            except Exception as e:
                messagebox.showerror("Erro ao salvar JSON", str(e))

        salvar_btn = tk.Button(editor_window, text="Salvar", command=salvar_alteracoes, font=SMALL_FONT)
        salvar_btn.grid(row=current_row, column=0, columnspan=2, pady=10)

    def gerar_hash_id(self):
        agora = datetime.now().strftime("%Y%m%d%H%M%S%f")  # AnoMesDiaHoraMinSegMicroseg
        return hex(abs(hash(agora)))[2:]  # Converte para hexadecimal e remove '0x'

    def load_sentence(self, delete_last_task_in_database=False):
        
        if delete_last_task_in_database:
            print(f"Deletar dados de tarefa no banco: {self.id_game_task}")
            del_game_task(id_game_task=self.id_game_task, game_name="word_shuffle_game")


        for widget in self.frame.winfo_children():
            widget.destroy()
        self.entries.clear()

        if not list_data_words:
            messagebox.showinfo("Fim", "Você completou todas as frases!")
            self.parent.quit()
            return

        self.dict_info_words = list_data_words.pop(random.randint(0, len(list_data_words) - 1))
        sentence = self.dict_info_words["text_eng"]
        sentence_question = self.dict_info_words["text_pt_br"]
        self.audio_path = self.dict_info_words["audio_path"]
        image_path = self.dict_info_words["image_figure"]

        sentence = sentence.translate(str.maketrans('', '', ',.?!')).lower()

        # Título da questão
        tk.Label(self.frame, text=sentence_question, font=("Arial", 14, "bold")).grid(row=0, column=0, columnspan=2, sticky="w", pady=10)

        # # Mostrar imagem (se existir)
        # if os.path.exists(image_path):
        #     img = Image.open(image_path).resize((150, 150))
        #     img = ImageTk.PhotoImage(img)
        #     self.img_label = tk.Label(self.frame, image=img)
        #     self.img_label.image = img
        #     self.img_label.grid(row=0, column=4, rowspan=6, padx=10)

        # Processar frase
        # header_frame = tk.Frame(self.frame)
        # header_frame.grid(row=0, column=0, columnspan=5, sticky="w", pady=(0, 5))

        # Cabeçalho alinhado com grid
        tk.Label(self.frame, text="Confirmar", font=("Arial", 10, "bold")).grid(row=1, column=0, padx=3, pady=3)
        tk.Label(self.frame, text="Desistir", font=("Arial", 10, "bold")).grid(row=1, column=1, padx=3, pady=3)
        tk.Label(self.frame, text="Palavra", font=("Arial", 10, "bold")).grid(row=1, column=2, padx=3, pady=3)
        tk.Label(self.frame, text="Embaralhada", font=("Arial", 10, "bold")).grid(row=1, column=3, padx=3, pady=3)
        tk.Label(self.frame, text="Acuracia", font=("Arial", 10, "bold")).grid(row=1, column=4, padx=3, pady=3)

        self.quantity_play_audio = 0
        words = sentence.strip().split()

        for i, word in enumerate(words):

            word_stats = self.word_stats_analyzer.get_word_info(word).to_dict("records")
            if word_stats:
                word_stats = word_stats[0]
            else:
                word_stats = {}

            shuffled = shuffle_word(word)

            check = tk.Button(self.frame, text="✅", command=lambda idx=i: self.check_single_answer(idx))
            check.grid(row=i + 2, column=0, padx=3, pady=2)

            give_up = tk.Button(self.frame, text="❌", command=lambda idx=i: self.give_up_word(idx))
            give_up.grid(row=i + 2, column=1, padx=3, pady=2)

            entry = tk.Entry(self.frame, width=15)
            entry.grid(row=i + 2, column=2, padx=3, pady=2)

            label = tk.Label(self.frame, text=shuffled)
            label.grid(row=i + 2, column=3, padx=3, pady=2, sticky="w")

            stats_acc = tk.Label(self.frame, text=word_stats.get("acuracia", "New Word!"))  # Exemplo
            stats_acc.grid(row=i + 2, column=4, padx=3, pady=2, sticky="w")

            clean_word = word.strip(string.punctuation).lower()
            self.entries.append({
                "word": word,
                "entry": entry,
                "clean_word": clean_word,
                "label": label,
                "check": check,
                "give_up": give_up,
                "stats_acc": stats_acc,
            })

            self.clicks_on_guess[i] = 0

        # self.submit_btn.config(state='normal')
        self.next_btn.config(state='disabled')

        self.id_game_task = self.gerar_hash_id()

    def save_game(self, word, idx, won):

        datetime_now = datetime.now().isoformat(timespec="seconds")
        
        path_word = Path(self.dict_info_words.get("path", None))

        category = path_word.parts[-3]
        sub_category = path_word.parts[-2]
        word_folder = path_word.parts[-1]

        game_data = {
            "id_game": self.id_game,
            "id_game_task": self.id_game_task,
            "datetime": datetime_now,
            "word": word,
            "category":category,
            "sub_category":sub_category,
            "word_folder":word_folder,
            # "hint": self.word_question,
            "won": won,
            # "used_attempts": attempts_used,
            "quantity_play_audio":self.quantity_play_audio,
            "clicks_on_guess": self.clicks_on_guess[idx],
            # "correct_guessed_letters": correct_guessed_letters,
            # "incorrect_guessed_letters": incorrect_guessed_letters,
            # "list_of_typed_letters":self.list_of_typed_letters,
            # "correct_guesses": correct_guesses,
            # "incorrect_guesses": incorrect_guesses,
            # "time_taken": time_taken,
            "game_name": "word_shuffle_game",
        }

        save_game_data(game_data)
        # print(game_data)
    
    def check_all_answers(self):
        all_correct = True
        for dict_entries in self.entries:
            user_input = dict_entries.get("entry").get().strip().lower()
            # print(user_input != correct_word)
            if user_input != dict_entries.get("clean_word"):
                all_correct = False

        if all_correct:
            pygame.mixer.music.stop()
            # messagebox.showinfo("Parabéns!", "Você acertou todas as palavras!")
            # self.submit_btn.config(state='disabled')
            self.next_btn.config(state='normal')
    
    def check_single_answer(self, idx):
        # entry, correct_word = self.entries[idx]
        self.clicks_on_guess[idx] += 1
        dict_entries = self.entries[idx]

        user_input = dict_entries.get("entry").get().strip().lower()
        if user_input == dict_entries.get("clean_word"):
            dict_entries.get("entry").config(bg="lightgreen")

            dict_entries.get("check").config(state='disabled')
            dict_entries.get("give_up").config(state='disabled')

            self.check_all_answers()

            self.save_game(word=dict_entries.get("word"), 
                           idx=idx, 
                           won=True)
        else:
            dict_entries.get("entry").config(bg="salmon")

    def give_up_word(self, idx):
        dict_entries = self.entries[idx]
        dict_entries.get("entry").delete(0, tk.END)
        dict_entries.get("entry").insert(0, dict_entries.get("clean_word"))
        dict_entries.get("entry").config(bg="yellow")


        dict_entries.get("check").config(state='disabled')
        dict_entries.get("give_up").config(state='disabled')

        self.check_all_answers()

        self.save_game(word=dict_entries.get("word"), 
                       idx=idx, 
                       won=False)


    def play_audio(self):
        if os.path.exists(self.audio_path):
            if pygame.mixer.music.get_busy():
                pygame.mixer.music.stop()
            pygame.mixer.music.load(self.audio_path)
            pygame.mixer.music.play()
            self.quantity_play_audio += 1
        else:
            messagebox.showerror("Erro", "Áudio não encontrado!")

# Iniciar app
if __name__ == "__main__":
    try:

        root = tk.Tk()
        root.title("Jogo da Forca com Imagens")
        game = WordShuffleGame(root)
        game.pack(expand=True, fill="both")
        root.title("Word Shuffle Game")
        root.geometry("640x520")
        root.mainloop()


        # root = tk.Tk()
        # app = WordShuffleGame(root)
        # root.mainloop()
    except Exception as e:
        print(f"Error {e}")
    finally:
        root.destroy()

TclError: can't invoke "destroy" command: application has been destroyed

In [37]:
{i:0 for i in range(3)}

{0: 0, 1: 0, 2: 0}

In [35]:
clicks_on_guess = {}

clicks_on_guess["a"] = 0

clicks_on_guess["a"] += 1

clicks_on_guess


{'a': 1}

In [26]:
import tkinter as tk

def highlight_letters(container, user_input, target_word):
    text_widget = tk.Text(container, height=1, width=len(target_word) + 1, borderwidth=0)
    text_widget.pack()

    for i, letter in enumerate(target_word):
        if i < len(user_input) and user_input[i] == letter:
            text_widget.insert("end", letter, "green")
        else:
            text_widget.insert("end", letter)
    
    text_widget.tag_config("green", foreground="green")
    text_widget.config(state="disabled")  # impede edição
    return text_widget

# Exemplo de uso
root = tk.Tk()
frame = tk.Frame(root)
frame.pack()

user_input = "t"  # Letra correta fornecida pelo usuário
target = "the"
highlight_letters(frame, user_input, target)

root.mainloop()


In [22]:
entries[0]

{'entry': 'entry', 'clean_word': 'clean_word', 'label': 'label'}

In [ ]:
end

In [ ]:
def shuffle_word(word):
    core = ''.join([c for c in word if c])
    if len(core) <= 1:
        return word
    shuffled = list(core)
    while True:
        random.shuffle(shuffled)
        if ''.join(shuffled) != core:
            break
    suffix = word[len(core):]
    return ''.join(shuffled) + suffix


for word in "It's a sharp pain.".split():
    shuffle = shuffle_word(word)
    print(shuffle)

tsI'
a
aprhs
np.ai


In [ ]:
word = "It's"

core = ''.join([c for c in word if c])

if len(core) <= 1:
    print(word)

shuffled = list(core)
while True:
    random.shuffle(shuffled)
    if ''.join(shuffled) != core:
        break
suffix = word[len(core):]

suffix, core

('', "It's")

In [ ]:
dict_info_words = list_data_words.pop(random.randint(0, len(list_data_words)-1))

dict_info_words#.get("text_eng", None)

{'path': 'database/extract_data_video/data/extracted_data/words/data_organize/verbos/expressão_de_sentimentos/to_feel',
 'files': "['text_v2.json', 'image_figure.jpg', 'image_text.jpg', 'audio.wav']",
 'text_pt_br': 'sentir',
 'text_eng': 'to feel',
 'audio_path': 'database/extract_data_video/data/extracted_data/words/data_organize/verbos/expressão_de_sentimentos/to_feel/audio.wav',
 'image_figure': 'database/extract_data_video/data/extracted_data/words/data_organize/verbos/expressão_de_sentimentos/to_feel/image_figure.jpg'}

In [ ]:
import os
import tkinter as tk
from tkinter import messagebox
from pathlib import Path
import pandas as pd
from PIL import Image, ImageTk
import json
import pygame
# import random
from datetime import datetime
from deep_translator import GoogleTranslator
# from random import randint

from app.utils.data_loader import DataLoader
from app.utils.game_timer import GameTimer
from app.utils.save_data import save_game_data

import numpy as np
from numpy import random

# np.random.seed(42)

# Inicializa o mixer do pygame
pygame.mixer.init()

TITLE_FONT = ("Arial", 20)
LABEL_FONT = ("Arial", 14)
SMALL_FONT = ("Arial", 12)

# DATA_PATH = Path("extract_data_video/data/extracted_data/words/data_organize")
DATA_PATH = Path("database/vocabulary/words/data_organize")
SUBCATEGORY_NAME = "em_loja"

class HangmanGame(tk.Frame):
    MAX_ATTEMPTS = 6

    def __init__(self, parent=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent
        # self.loader = DataLoader(base_path=DATA_PATH)
        # self.list_data_words:list[dict] = self.loader.get_all_words(subcategory_name=SUBCATEGORY_NAME)

        self.list_data_words:list[dict] = self.open_json('database/vocabulary/study_word_list.json')
        random.shuffle(self.list_data_words)

        self.timer = GameTimer(self)

        self.clicks_on_guess = 0

        self.last_word = None

        self.id_game = self.gerar_hash_id()
        self.reset_game()
        self.setup_ui()

    def gerar_hash_id(self):
        agora = datetime.now().strftime("%Y%m%d%H%M%S%f")  # AnoMesDiaHoraMinSegMicroseg
        return hex(abs(hash(agora)))[2:]  # Converte para hexadecimal e remove '0x'

    def reset_game(self):
        if not self.list_data_words:
            messagebox.showinfo("Fim", "Não há mais palavras disponíveis.")
            self.parent.quit()
            return

        self.list_of_typed_letters = []
        self.dict_info_words = self.list_data_words.pop(random.randint(0, len(self.list_data_words)-1))

        self.word_answer = self.dict_info_words.get("text_eng", None)
        self.word_answer = self.word_answer.lower()
        
        self.word_question = self.dict_info_words.get("text_pt_br", None) 

        self.guessed_word = self.hide_text(self.word_answer) 
        self.remaining_attempts = self.MAX_ATTEMPTS
        self.guessed_letters = set()

        self.timer.reset_timer()
        self.timer.play()

    def hide_text(self, text: str) -> str:
        # return [" " if c == " " else "_" for c in text]
        return ['_' if c.isalpha() else c for c in text]

    def show_label_words(self):
        font=("Arial", 12)
        fg="black"
        self.palavras_restantes_label = tk.Label(self, text=f"Palavras restantes: {len(self.list_data_words)}", font=font, fg=fg)
        self.palavras_restantes_label.place(relx=0, rely=0.0, anchor="nw")

    def setup_ui(self):

        self.show_label_words()

        self.word_label = tk.Label(self, text=" ".join(self.guessed_word), font=TITLE_FONT)
        self.word_label.pack(pady=10)

        self.attempts_label = tk.Label(self, text="", font=LABEL_FONT)
        self.attempts_label.pack()

        self.guessed_label = tk.Label(self, text="", font=SMALL_FONT)
        self.guessed_label.pack()

        self.image_label = tk.Label(self)
        self.image_label.pack(pady=10)

        self.hint_label = tk.Label(self, text="", font=LABEL_FONT)
        self.hint_label.pack()

        self.input_frame = tk.Frame(self)
        self.input_frame.pack(pady=10)

        tk.Label(self.input_frame, text="Letra:", font=SMALL_FONT).pack(side="left")
        self.letter_entry = tk.Entry(self.input_frame, width=5, font=LABEL_FONT)
        self.letter_entry.pack(side="left")

        self.guess_button = tk.Button(self.input_frame, text="Adivinhar", command=self.check_guess)
        self.guess_button.pack(side="left", padx=5)

        self.btn_editar = tk.Button(self, text="✏️ Editar")
        self.btn_editar.config(command=lambda: self.editar_json(dict_info_words=self.dict_info_words))
        self.btn_editar.place(relx=0.0, rely=0.5, anchor="w")

        self.btn_editar = tk.Button(self, text="🎲 Embaralhar")
        self.btn_editar.config(command=self.restart_game)
        self.btn_editar.place(relx=0.0, rely=0.56, anchor="w")

        self.update_ui()


    def update_ui(self):
        
        self.palavras_restantes_label.config(text=f"Palavras restantes: {len(self.list_data_words)}")

        self.word_label.config(text=" ".join(self.guessed_word))
        self.attempts_label.config(text=f"Tentativas restantes: {self.remaining_attempts}")
        self.guessed_label.config(text=f"Letras tentadas: {', '.join(sorted(self.guessed_letters))}")
        self.hint_label.config(text=f"Dica: {self.word_question}")


        image_figure = self.dict_info_words.get("image_figure", None)

        if image_figure:
            img_path = image_figure
            img = Image.open(img_path).resize((200, 200))
            photo = ImageTk.PhotoImage(img)

            self.image_label.config(image=photo)
            self.image_label.image = photo  # <- MANTÉM REFERÊNCIA
        else:
            self.image_label.config(image='')
            self.image_label.image = None

    def check_guess(self):
        self.clicks_on_guess += 1

        guess = self.letter_entry.get().strip().lower()
        self.letter_entry.delete(0, tk.END)

        if not guess.isalpha():
            messagebox.showwarning("Letra inválida", "Digite apenas letras.")
            return

        # Repedindo para garantir que todas as letras sejam adicionadas
        for letter in guess:
            self.list_of_typed_letters.append(letter.lower())
            

        for letter in guess:
            if letter in self.guessed_letters:
                continue

            self.guessed_letters.add(letter)

            if letter in self.word_answer:
                self.reveal_letters(letter)
            else:
                self.remaining_attempts -= 1

            if self.check_game_end():
                return

        self.update_ui()
    
    def editar_json(self, dict_info_words):
        SMALL_FONT = ("Arial", 10)  # ou use a fonte que desejar
        path_base = Path(dict_info_words.get("path", ""))
        path_text_json = path_base / "text_v2.json"
        path_image = path_base / "image_text.jpg"

        if not path_text_json.exists():
            messagebox.showerror("Erro", f"Arquivo não encontrado: {path_text_json}")
            return

        try:
            with open(path_text_json, 'r', encoding="utf-8") as f:
                json_data = json.load(f)
        except Exception as e:
            messagebox.showerror("Erro ao abrir JSON", str(e))
            return

        # Cria nova janela
        editor_window = tk.Toplevel(self)
        editor_window.title("Editar texto")

        current_row = 0

        # Exibe imagem no topo (se existir)
        if path_image.exists():
            try:
                img = Image.open(path_image).resize((200, 200))
                photo = ImageTk.PhotoImage(img)

                img_label = tk.Label(editor_window, image=photo)
                img_label.image = photo  # evitar que o garbage collector apague
                img_label.grid(row=current_row, column=0, columnspan=2, pady=10)
                current_row += 1
            except Exception as e:
                messagebox.showwarning("Imagem", f"Erro ao carregar imagem: {e}")

        entries = {}

        for key, value in json_data.items():
            tk.Label(editor_window, text=key, font=SMALL_FONT).grid(row=current_row, column=0, padx=10, pady=5, sticky="e")
            entry = tk.Entry(editor_window, width=60)
            entry.insert(0, value)
            entry.grid(row=current_row, column=1, padx=10, pady=5, sticky="w")
            entries[key] = entry
            current_row += 1

        def salvar_alteracoes():
            for key in json_data:
                json_data[key] = entries[key].get()

            try:
                with open(path_text_json, 'w', encoding="utf-8") as f:
                    json.dump(json_data, f, ensure_ascii=False, indent=4)
                messagebox.showinfo("Sucesso", "Arquivo salvo com sucesso.")
                editor_window.destroy()
            except Exception as e:
                messagebox.showerror("Erro ao salvar JSON", str(e))

        salvar_btn = tk.Button(editor_window, text="Salvar", command=salvar_alteracoes, font=SMALL_FONT)
        salvar_btn.grid(row=current_row, column=0, columnspan=2, pady=10)

    def reveal_letters(self, letter):
        for idx, char in enumerate(self.word_answer):
            if char == letter:
                self.guessed_word[idx] = letter

    def exibir_fim_de_jogo(self):
        # Cria uma nova janela
        fim_window = tk.Toplevel(self)
        fim_window.title("Fim de jogo")
        fim_window.grab_set()  # modal
        fim_window.configure(bg="white")

        # Ícone de erro
        icon_label = tk.Label(fim_window, text="❌", font=("Arial", 40), bg="white", fg="red")
        icon_label.pack(pady=(15, 0))

        # Mensagem da palavra correta
        msg_label = tk.Label(
            fim_window,
            text=f"A palavra era: {self.word_answer}",
            font=("Arial", 12, "bold"),
            bg="white",
            fg="black",
            wraplength=300,
            justify="center"
        )
        msg_label.pack(padx=20, pady=10)

        # Frame para os botões
        btn_frame = tk.Frame(fim_window, bg="white")
        btn_frame.pack(pady=(0, 15))

        # Botão Editar
        btn_editar = tk.Button(
            btn_frame,
            text="✏️ Editar",
            font=("Arial", 10),
            width=15,
            command=lambda: [fim_window.destroy(), self.editar_json(self.last_word)]
        )
        btn_editar.pack(side="left", padx=10)

        # Botão OK
        btn_ok = tk.Button(
            btn_frame,
            text="✅ OK",
            font=("Arial", 10),
            width=15,
            command=fim_window.destroy
            # command=lambda: [fim_window.destroy(), self.restart_game()]
        )
        btn_ok.pack(side="left", padx=10)

        # Centraliza a janela
        fim_window.update_idletasks()
        w = fim_window.winfo_width()
        h = fim_window.winfo_height()
        x = (fim_window.winfo_screenwidth() // 2) - (w // 2)
        y = (fim_window.winfo_screenheight() // 2) - (h // 2)
        fim_window.geometry(f"{w}x{h}+{x}+{y}")

    def exibir_vitoria(self):
        vitoria_window = tk.Toplevel(self)
        vitoria_window.title("Você venceu!")
        vitoria_window.grab_set()
        vitoria_window.configure(bg="white")

        # Ícone e mensagem
        icon_label = tk.Label(vitoria_window, text="🎉", font=("Arial", 40), bg="white", fg="green")
        icon_label.pack(pady=(15, 0))

        msg_label = tk.Label(
            vitoria_window,
            text=f"Você venceu!\nA palavra era: {self.word_answer}",
            font=("Arial", 12, "bold"),
            bg="white",
            fg="black",
            wraplength=300,
            justify="center"
        )
        msg_label.pack(padx=20, pady=10)

        # Frame para os botões
        btn_frame = tk.Frame(vitoria_window, bg="white")
        btn_frame.pack(pady=(0, 15))

        # Botão Editar
        btn_editar = tk.Button(
            btn_frame,
            text="✏️ Editar",
            font=("Arial", 10),
            width=15,
            command=lambda: [vitoria_window.destroy(), self.editar_json(self.last_word)]
        )
        btn_editar.pack(side="left", padx=10)

        # Botão OK
        btn_ok = tk.Button(
            vitoria_window,
            text="OK",
            font=("Arial", 10),
            width=15,
            command=vitoria_window.destroy
            # command=lambda: [vitoria_window.destroy(), self.restart_game()]
        )
        btn_ok.pack(pady=(0, 15))

        # Centraliza
        vitoria_window.update_idletasks()
        w = vitoria_window.winfo_width()
        h = vitoria_window.winfo_height()
        x = (vitoria_window.winfo_screenwidth() // 2) - (w // 2)
        y = (vitoria_window.winfo_screenheight() // 2) - (h // 2)
        vitoria_window.geometry(f"{w}x{h}+{x}+{y}")

    def check_game_end(self) -> bool:
        if "_" not in self.guessed_word:
            self.save_score(True)
            # messagebox.showinfo("Você venceu!", f"A palavra era: {self.word_answer}")
            # self.restart_game()
            self.exibir_vitoria()
            self.restart_game()
            return True

        if self.remaining_attempts <= 0:
            self.save_score(False)

            # Coloca a palavra de volta na lista
            self.list_data_words.append(self.dict_info_words)
            
            # messagebox.showerror("Fim de jogo", f"A palavra era: {self.word_answer}")
            self.exibir_fim_de_jogo()
            self.restart_game()
            return True

        return False

    def restart_game(self):
        self.last_word = self.dict_info_words.copy()
        self.reset_game()
        self.update_ui()

    def save_score(self, won: bool):
        attempts_used = self.MAX_ATTEMPTS - self.remaining_attempts
        difficulty = attempts_used / self.MAX_ATTEMPTS

        correct_guessed_letters=[letter for letter in self.guessed_word if letter.isalnum()]
        incorrect_guessed_letters = list(set(self.guessed_letters).difference(set(self.guessed_word)))

        # finaliza_tempo
        # end_time = time.time()
        time_taken = self.timer.elapsed_time

        clicks_on_guess = self.clicks_on_guess

        path_word = Path(self.dict_info_words.get("path", None))

        category = path_word.parts[-3]
        sub_category = path_word.parts[-2]
        # adjetivos/sobre_as_pessoas/unable

        # if not any([category == loader_category.name for loader_category in self.loader.get_categories()]):
        #     print("Categoria Não Existe")

        self.timer.reset_timer()
        self.clicks_on_guess = 0

        # assert total_attempts >= used_attempts, "Tentativas usadas não podem ser maiores que o total permitido."

        datetime_now = datetime.now().isoformat(timespec="seconds")
        

        game_data = {
            "id_game": self.id_game,
            "datetime": datetime_now,
            "word": self.word_answer,
            "category":category,
            "sub_category":sub_category,
            "hint": self.word_question,
            "won": won,
            "difficulty": difficulty,
            "total_attempts": self.MAX_ATTEMPTS,
            "used_attempts": attempts_used,
            "clicks_on_guess": clicks_on_guess,
            "correct_guessed_letters": correct_guessed_letters,
            "incorrect_guessed_letters": incorrect_guessed_letters,
            "list_of_typed_letters":self.list_of_typed_letters,
            # "correct_guesses": correct_guesses,
            # "incorrect_guesses": incorrect_guesses,
            "time_taken": time_taken,
            "game_name": "hangman",
        }

        save_game_data(game_data)
        # self.save_score()

    def translate_sentence(self, sentence, source_lang="en", target_lang="pt"):
        try:
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            return translator.translate(sentence)
        except Exception as e:
            print(f"Translation error: {e}")
            return "Translation unavailable"
    
    def open_json(self, filepath):
        try:
            with open(filepath, 'r', encoding="utf-8") as file:
                data = json.load(file)
                return data
        except FileNotFoundError:
            print(f"Error: File not found at {filepath}")
            return None
        except json.JSONDecodeError:
            print(f"Error: Invalid JSON format in {filepath}")
            return None

if __name__ == "__main__":
    try:
        root = tk.Tk()
        root.title("Jogo da Forca com Imagens")
        game = HangmanGame(root)
        game.pack(expand=True, fill="both")
        root.geometry("600x600")
        root.mainloop()
    except Exception as e:
        print(f"Error {e}")
    
    finally:
        root.destroy()


TclError: can't invoke "destroy" command: application has been destroyed